<h1> Data Loading </h1>

In [1]:
import pandas as pd

In [2]:
data_corr=pd.read_csv("apache-ant-corr.csv")

In [3]:
data_corr[:5]

,ID,projectname,classification,Abstract,label
0,0,apache-ant-1.7.0,DEFECT,// FIXME formatters are not thread-safe,yes
1,1,apache-ant-1.7.0,DEFECT,"// XXX: (Jon Skeet) The comment ""if it hasn't ...",yes
2,2,apache-ant-1.7.0,DEFECT,"// I hate to admit it, but we don't know what ...",yes
3,3,apache-ant-1.7.0,DEFECT,// Just a note: StarTeam has a status for NEW ...,yes
4,4,apache-ant-1.7.0,DEFECT,// the generated classes must not be added in ...,yes


In [4]:
data_corr.shape

(4098, 5)

In [5]:
data_corr.loc[data_corr.label=='yes', 'label'].count()
#(data_corr.label=='yes').sum()

135

In [6]:
data_corr.query('label == "no"').label.count()
#data_corr.label.value_counts().no  
#Note value_counts() is used only with series type of data

3963

In [7]:
comments=data_corr[['Abstract']]

In [8]:
comments[:5]

,Abstract
0,// FIXME formatters are not thread-safe
1,"// XXX: (Jon Skeet) The comment ""if it hasn't ..."
2,"// I hate to admit it, but we don't know what ..."
3,// Just a note: StarTeam has a status for NEW ...
4,// the generated classes must not be added in ...


In [9]:
labels=data_corr[['label']]

In [10]:
labels[:5]

,label
0,yes
1,yes
2,yes
3,yes
4,yes


In [15]:
labels_list=[]
for i, row in labels.iterrows():
    labels_list.append(row['label'])
    
comments_list=[]
for i, row in comments.iterrows():
    comments_list.append(row['Abstract'])

<h1> Start modelling </h1>

In [25]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np

In [44]:
from glob import glob
import random
import os
from sklearn.preprocessing import LabelEncoder
import numpy as np


In [57]:
def data_preprocessing(num_words, sequence_length, test_size=0.25, oov_token=None):
    
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
    tokenizer.fit_on_texts(comments_list)
    X = tokenizer.texts_to_sequences(comments_list)
    X,l= np.array(X), np.array(labels)
    
    X = pad_sequences(X, maxlen=sequence_length)
    label_encoder = LabelEncoder()
    vec = label_encoder.fit_transform(l)
    y=to_categorical(vec)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)
    data = {}
    data["X_train"] = X_train
    data["X_test"]= X_test
    data["y_train"] = y_train
    data["y_test"] = y_test
    data["tokenizer"] = tokenizer
    data["int2label"] =  {0: "no", 1: "yes"}
    data["label2int"] = {"no": 0, "yes": 1}
    
    
    return data

<h1> Build Embeddings </h1>

In [58]:
def get_embedding_vectors(word_index, embedding_size=100):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_size))
    with open(f"data/glove.6B.{embedding_size}d.txt", encoding="utf8") as f:
        for line in tqdm(f, "Reading GloVe"):
            values = line.split()
            # get the word as the first word in the line
            word = values[0]
            if word in word_index:
                idx = word_index[word]
                # get the vectors as the remaining values in the line
                embedding_matrix[idx] = np.array(values[1:], dtype="float32")
    return embedding_matrix

In [59]:
def create_model(word_index, units=128, n_layers=1, cell=LSTM, bidirectional=False,
                embedding_size=100, sequence_length=100, dropout=0.3, 
                loss="categorical_crossentropy", optimizer="adam", 
                output_length=2):
    """Constructs a RNN model given its parameters"""
    embedding_matrix = get_embedding_vectors(word_index, embedding_size)
    model = Sequential()
    # add the embedding layer
    model.add(Embedding(len(word_index) + 1,
              embedding_size,
              weights=[embedding_matrix],
              trainable=False,
              input_length=sequence_length))
    for i in range(n_layers):
        if i == n_layers - 1:
            # last layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=False)))
            else:
                model.add(cell(units, return_sequences=False))
        else:
            # first layer or hidden layers
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True)))
            else:
                model.add(cell(units, return_sequences=True))
        model.add(Dropout(dropout))
    model.add(Dense(output_length, activation="softmax"))
    # compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
    return model

In [60]:
# max number of words in each sentence
SEQUENCE_LENGTH = 300
# N-Dimensional GloVe embedding vectors
EMBEDDING_SIZE = 300
# number of words to use, discarding the rest
N_WORDS = 10000
# out of vocabulary token
OOV_TOKEN = None
# 30% testing set, 70% training set
TEST_SIZE = 0.3
# number of CELL layers
N_LAYERS = 1
# the RNN cell to use, LSTM in this case
RNN_CELL = LSTM
# whether it's a bidirectional RNN
IS_BIDIRECTIONAL = False
# number of units (RNN_CELL ,nodes) in each layer
UNITS = 128
# dropout rate
DROPOUT = 0.4
### Training parameters
LOSS = "categorical_crossentropy"
OPTIMIZER = "adam"
BATCH_SIZE = 64
EPOCHS = 6

def get_model_name(dataset_name):
    # construct the unique model name
    model_name = f"{dataset_name}-{RNN_CELL.__name__}-seq-{SEQUENCE_LENGTH}-em-{EMBEDDING_SIZE}-w-{N_WORDS}-layers-{N_LAYERS}-units-{UNITS}-opt-{OPTIMIZER}-BS-{BATCH_SIZE}-d-{DROPOUT}"
    if IS_BIDIRECTIONAL:
        # add 'bid' str if bidirectional
        model_name = "bid-" + model_name
    if OOV_TOKEN:
        # add 'oov' str if OOV token is specified
        model_name += "-oov"
    return model_name

In [63]:
# create these folders if they does not exist
if not os.path.isdir("results"):
    os.mkdir("results")
if not os.path.isdir("logs"):
    os.mkdir("logs")
if not os.path.isdir("data"):
    os.mkdir("data")
# load the data
data = data_preprocessing(N_WORDS, SEQUENCE_LENGTH, TEST_SIZE, oov_token=OOV_TOKEN)
dataset_name = "apache-ant"
model_name = get_model_name(dataset_name)

# construct the model
model = create_model(data["tokenizer"].word_index, units=UNITS, n_layers=N_LAYERS, 
                    cell=RNN_CELL, bidirectional=IS_BIDIRECTIONAL, embedding_size=EMBEDDING_SIZE, 
                    sequence_length=SEQUENCE_LENGTH, dropout=DROPOUT, 
                    loss=LOSS, optimizer=OPTIMIZER, output_length=data["y_train"][0].shape[0])
model.summary()

# using tensorboard on 'logs' folder
tensorboard = TensorBoard(log_dir=os.path.join("logs", model_name))
# start training
history = model.fit(data["X_train"], data["y_train"],
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(data["X_test"], data["y_test"]),
                    callbacks=[tensorboard],
                    verbose=1)
# save the resulting model into 'results' folder
model.save(os.path.join("results", model_name) + ".h5")

Reading GloVe: 400000it [00:24, 16045.14it/s]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          1151400   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               219648    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 1,371,306
Trainable params: 219,906
Non-trainable params: 1,151,400
_________________________________________________________________
Epoch 1/6
 1/45 [..............................] - ETA: 0s - loss: 0.7012 - accuracy: 0.5312WARNING:tensorflow:From C:\Users\Ketaki Barde\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (fr

In [64]:
def get_predictions(text):
    sequence = data["tokenizer"].texts_to_sequences([text])
    # pad the sequences
    sequence = pad_sequences(sequence, maxlen=SEQUENCE_LENGTH)
    # get the prediction
    prediction = model.predict(sequence)[0]
    return prediction, data["int2label"][np.argmax(prediction)]

In [66]:
text = "//DMS handle this well."
output_vector, prediction = get_predictions(text)
print("Output vector:", output_vector)
print("Prediction:", prediction)

Output vector: [0.9916232  0.00837673]
Prediction: no


In [67]:
text = "// TODO fix the space above."
output_vector, prediction = get_predictions(text)
print("Output vector:", output_vector)
print("Prediction:", prediction)

Output vector: [0.36631143 0.63368857]
Prediction: yes
